# 概要

Juliaとawabi.jlによる形態素解の実験

## データの取得

In [1]:
ENV["GKS_ENCODING"] = "utf-8"
using HTTP, JSON, CSV, TableOperations, DataFrames, LightXML, Plots, Measures

"""
HTTPでデータを取得する関数
"""
function api_get(url)
  count = 0
  while count < 10
    try
      response = HTTP.get(url)
      response.status == 200 && (
        return String(response.body);
      )
    finally
      count += 1
    end
  end
  return error("Error: bad url")
end

"""
ニコニコ動画のデータを収集しファイルに保存する関数

参考: https://dwango.github.io/niconico/genre_ranking/ranking_log
"""
function collect_nico_data(ranking_type, date, file_type, endpoint="https://dcdn.cdn.nimg.jp/nicovideo/old-ranking")
  url       = "$(endpoint)/$(ranking_type)/$(date)/$(file_type)"
  file_name = replace(url, "/" => "_")
  write(file_name, api_get(url))
  return file_name
end

"""
ファイル名リストからタグを検索し、ファイル名を返す関数
"""
function search_tag(tag, file_name)
  file_data_list = JSON.parsefile(file_name)
  for file_data ∈ file_data_list
    file_data["tag"] == tag && (
      return file_data["file"]
    )
  end
  return error("Error: $(tag) is not found")
end

"""
ファイル名からデータを集計し、タグごとのデータを返す関数
"""
function aggregate_by_tag(ac6_result_file)

  function get_tags_by_id(id)
    xml_element = "https://ext.nicovideo.jp/api/getthumbinfo/$(id)" |> api_get |> parse_string |> root
    for class ∈ ["thumb", "tags"]
      xml_element = find_element(xml_element, class)
      isnothing(xml_element) && (
        #=
        ここに入るときは削除されてるかも
        <nicovideo_thumb_response status="fail">
          <error>
            <code>DELETED</code>
            <description>deleted</description>
          </error>
        </nicovideo_thumb_response>
        =#
        return [];
      )
    end
    elements = [content(e) for e ∈ get_elements_by_tagname(xml_element, "tag")]
  end

  tag_data         = Dict{String,Dict{String,Int64}}()
  tag_data_by_view = Dict{String,Dict{String,Float64}}()
  ac6_results      = JSON.parsefile(ac6_result_file)
  AGGREGATE_COLS   = ["view", "like", "comment", "mylist"]
  tag_unique_set   = Set{String}()
  for ac6_result ∈ ac6_results
    tags = get_tags_by_id(ac6_result["id"])
    for tag ∈ tags
      push!(tag_unique_set, tag)
      haskey(tag_data, tag) || (
        tag_data[tag]         = Dict(col => 0 for col ∈ AGGREGATE_COLS);
        tag_data_by_view[tag] = Dict(col => 0 for col ∈ AGGREGATE_COLS);
      )
      for col ∈ AGGREGATE_COLS
        tag_data[tag][col] += ac6_result["count"][col]
      end
    end
  end
  for tag ∈ tag_unique_set
    for col ∈ ["like", "comment", "mylist"]
      tag_data_by_view[tag][col] = tag_data[tag][col] * 1000 / tag_data[tag]["view"]
    end
  end
  write("aggregate_$(ac6_result_file)", JSON.json(tag_data))
  write("aggregate_by_view_$(ac6_result_file)", JSON.json(tag_data_by_view))
  return tag_data_by_view
end

aggregate_by_tag

In [33]:
file_name_list_name = collect_nico_data("daily", "2023-09-23", "file_name_list.json")
ac6_file_name       = search_tag("アーマードコア6", file_name_list_name)
ac6_result_file     = collect_nico_data("daily", "2023-09-23", ac6_file_name)
tag_data_by_view    = aggregate_by_tag(ac6_result_file)

Dict{String, Dict{String, Float64}} with 497 entries:
  "acvi対戦動画"           => Dict("view"=>0.0, "like"=>22.7683, "comment"=>68.…
  "集団幻覚"               => Dict("view"=>0.0, "like"=>25.6461, "comment"=>4.6…
  "ストライダー"           => Dict("view"=>0.0, "like"=>48.8431, "comment"=>23.…
  "seirenvoice"            => Dict("view"=>0.0, "like"=>52.7236, "comment"=>48.…
  "坊やよいこだねんねしな" => Dict("view"=>0.0, "like"=>61.8375, "comment"=>78.…
  "人格の重量過多"         => Dict("view"=>0.0, "like"=>21.6482, "comment"=>58.…
  "Voiceroid実況プレイ"    => Dict("view"=>0.0, "like"=>179.487, "comment"=>487…
  "玉入れ"                 => Dict("view"=>0.0, "like"=>16.4542, "comment"=>32.…
  "初投稿動画"             => Dict("view"=>0.0, "like"=>102.041, "comment"=>52.…
  "cevio_ai"               => Dict("view"=>0.0, "like"=>181.818, "comment"=>120…
  "大日本帝国版"           => Dict("view"=>0.0, "like"=>72.8155, "comment"=>97.…
  "谷村ブラックタンク"     => Dict("view"=>0.0, "like"=>16.4542, "comment"=>32.…
  "様子のおかしい人です"   => Dict("view"=>0

In [109]:
"""
視聴回数あたりのいいね数、コメント数、マイリスト数を計算し、視覚化する
data_type: "julia_dict" or "json_file"
"""
function plot_data_by_view(tag_data_by_view, data_type = "julia_dict")

  data_type == "json_file" && (
    cd(@__DIR__());
    tag_data_by_view = JSON.parsefile(tag_data_by_view);
  )

  gr(fontfamily="JuliaMono")
  # タイトルとリアクション割合の棒グラフを作成する
  df         = DataFrame()
  df.tag     = collect(keys(tag_data_by_view))
  df.view    = [tag_data_by_view[t]["view"]    for t in df.tag]
  df.mylist  = [tag_data_by_view[t]["mylist"]  for t in df.tag]
  df.like    = [tag_data_by_view[t]["like"]    for t in df.tag]
  df.comment = [tag_data_by_view[t]["comment"] for t in df.tag]
  df         = sort(df, :view, rev=false)
  df         = df[end-41:end-1, :]

  # 棒グラフを作成する
  my_bar(x,y,str,n) = bar(
    x,
    y,
    fontfamily  = "JuliaMono",
    orientation = "horizontal",
    yticks      = (0.5:length(x) , x .|> (t -> t * " "^(13) )),
    xticks      = (0:n:maximum(y), string.(Int.(collect(0:n:maximum(y))))),
    title       = "\nアーマードコア6 タグ毎の視聴回数1000回あたりの$(str)数                              ",
    size        = (2000, 1350),
    bar_width   = 0.7,
    left_margin = 10mm,
    right_margin= 5mm,
    widen       = true,
    legend      = false,
    ylimit      = (0, length(df.tag) + 1),
  )
  
  plot(
  my_bar(df.tag, df.mylist, "マイリスト", 5),
  # savefig("ac6_mylist_per_view.svg")
  my_bar(df.tag, df.like, "いいね", 10),
  # savefig("ac6_like_per_view.svg")
  my_bar(df.tag, df.comment, "コメント", 20),
  # savefig("ac6_comment_per_view.svg")
  )
  savefig("ac6_reaction_per_view.svg")
end
# plot_data_by_view(tag_data_by_view)
plot_data_by_view("aggregate_by_view_https:__dcdn.cdn.nimg.jp_nicovideo_old-ranking_daily_2023-09-23_game_04.json", "json_file")

# plot_data_by_view("aggregate_https:__dcdn.cdn.nimg.jp_nicovideo_old-ranking_daily_2023-09-23_game_04.json", "json_file")

"/home/izumi/git/analysis_tools/nicovideo_aggregate/experimental/ac6_reaction_per_view.svg"

"utf8"

In [52]:
@show df = dataset(DataFrame, "tips")

df = dataset(DataFrame, "tips") = 244×7 DataFrame
 Row │ total_bill  tip      sex      smoker   day      time     size
     │ Float64     Float64  String7  String3  String7  String7  Int64
─────┼────────────────────────────────────────────────────────────────
   1 │      16.99     1.01  Female   No       Sun      Dinner       2
   2 │      10.34     1.66  Male     No       Sun      Dinner       3
   3 │      21.01     3.5   Male     No       Sun      Dinner       3
   4 │      23.68     3.31  Male     No       Sun      Dinner       2
   5 │      24.59     3.61  Female   No       Sun      Dinner       4
   6 │      25.29     4.71  Male     No       Sun      Dinner       4
   7 │       8.77     2.0   Male     No       Sun      Dinner       2
   8 │      26.88     3.12  Male     No       Sun      Dinner       4
   9 │      15.04     1.96  Male     No       Sun      Dinner       2
  10 │      14.78     3.23  Male     No       Sun      Dinner       2
  11 │      10.27     1.71  Male     No 

Row,total_bill,tip,sex,smoker,day,time,size
,Float64,Float64,String7,String3,String7,String7,Int64
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.5,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4
6,25.29,4.71,Male,No,Sun,Dinner,4
7,8.77,2.0,Male,No,Sun,Dinner,2
8,26.88,3.12,Male,No,Sun,Dinner,4
9,15.04,1.96,Male,No,Sun,Dinner,2


In [5]:

using Pkg
Pkg.status()

Status `~/git/analysis_tools/Project.toml`
  [b89ecf66] Awabi v0.1.3
  [336ed68f] CSV v0.10.11
  [a93c6f00] DataFrames v1.6.1
⌃ [c43c736e] Genie v5.2.2
⌅ [cd3eb016] HTTP v0.9.17
  [7073ff75] IJulia v1.24.2
  [682c06a0] JSON v0.21.4
  [9c8b4983] LightXML v0.9.0
  [58dd65bb] Plotly v0.4.1
  [91a5bcdd] Plots v1.39.0
  [8162dcfd] PrettyPrint v0.2.0
  [ab02a1b2] TableOperations v1.2.0
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [3]:
cd(@__DIR__()*"/../..")
pwd()
using Pkg
Pkg.activate(".")
cd(@__DIR__())


  Activating project at `~/git/analysis_tools`
